In [ ]:
# Packages, seed and path
## packages
from edge_sim_py import *
import math
import os
import random
import msgpack
import pandas as pd
import matplotlib.pyplot as plt

## seed
import torch
torch.manual_seed(5)
import random
random.seed(5)
import numpy as np
np.random.seed(5)

## path
algo_name = "worstfit_lm"

In [ ]:
# Functions
# def custom_collect_method(self) -> dict: # Custom collect method to measure the power consumption of each server
#     metrics = {
#         "Instance ID": self.id,
#         "Power Consumption": self.get_power_consumption(),
#     }
#     return metrics

power_list = list() # List to store total power consumption everytime the task scheduling algorithm is used


def my_algorithm(parameters):
    
    print("\n\n")
    total_power = 0 #We sum the power consumption after migrating each service
    for service in Service.all(): #Iterate over every service
        
 
        if not service.being_provisioned: #If service needs to be migrated

            
            #To sort edge servers based on their available free resources, we will use Python's "sorted" method. 
            #The server capacity is represented by three layers: CPU, memory, and disk. To determine the average resource utilization of each server, 
            #we calculate the geometric mean of these three layers. Finally, we set the "reverse" attribute of the sorted method to "True," 
            #allowing us to arrange the edge servers in descending order of their free resources.


            edge_servers = sorted(
                EdgeServer.all(),
                key=lambda s: ((s.cpu - s.cpu_demand) * (s.memory - s.memory_demand) * (s.disk - s.disk_demand)) ** (1 / 3),
                reverse=True,
            )

            for edge_server in edge_servers:
                # Check if the edge server has resources to host the service
                if edge_server.has_capacity_to_host(service=service):
                    # We just need to migrate the service if it's not already in the least occupied edge server
                    if service.server != edge_server:
                        print(f"[STEP {parameters['current_step']}] Migrating {service} From {service.server} to {edge_server}")
                        
                        service.provision(target_server=edge_server)

                        
                        #get the sum of powerconsumption of each edge server
                        power = 0

                        for iter_edge_server in EdgeServer.all():
                            power = power + iter_edge_server.get_power_consumption()
                        
                        #Add power consumption after migrating current service
                        total_power += power
                        
                        # After start migrating the service we can move on to the next service
                        break

    #Append to power_list for plotting
    power_list.append(total_power)


def stopping_criterion(model: object):    
    # As EdgeSimPy will halt the simulation whenever this function returns True,
    # its output will be a boolean expression that checks if the current time step is 600
    return model.schedule.steps == 1000

In [ ]:
# Simulation execution
simulator = Simulator(
    tick_duration=1,
    tick_unit="seconds",
    stopping_criterion=stopping_criterion,
    resource_management_algorithm=my_algorithm,
)

# Loading a sample dataset
#simulator.initialize(input_file="sample_dataset3.json")
simulator.initialize(input_file="https://raw.githubusercontent.com/EdgeSimPy/edgesimpy-tutorials/master/datasets/sample_dataset2.json")

#Assigning the custom collect method
#EdgeServer.collect = custom_collect_method

# Executing the simulation
simulator.run_model()

In [ ]:
# Results
## Retrieving logs dataframe for plot
logs_containerregistry = pd.DataFrame(simulator.agent_metrics["ContainerRegistry"])
logs_edgeserver = pd.DataFrame(simulator.agent_metrics["EdgeServer"])
logs_networkflow = pd.DataFrame(simulator.agent_metrics["NetworkFlow"])
logs_networkswitch = pd.DataFrame(simulator.agent_metrics["NetworkSwitch"])
logs_service = pd.DataFrame(simulator.agent_metrics["Service"])
logs_user = pd.DataFrame(simulator.agent_metrics["User"])

In [ ]:
logs_edgeserver['CPU Usage'] = (logs_edgeserver['CPU Demand']*100)/logs_edgeserver['CPU']
logs_edgeserver['RAM Usage'] = (logs_edgeserver['RAM Demand']*100)/logs_edgeserver['RAM']
logs_edgeserver[['Object', 'Power Consumption', 'CPU Usage', 'RAM Usage']].groupby(by=['Object']).mean()#.mean()

In [ ]:
logs_edgeserver